In [ ]:
import numpy as np
import time

# Students will submit their files with their team-name.py
# Student have to use the Team as their parent class

In [ ]:
class Team:
    def __init__(self, self_features, opponent_features, environment):
        self.self_features = self_features
        self.opponent_features = opponent_features
        self.explore_wicket = 0
        self.explore_runs = 0
        self.environment = environment

    def get_next_batter(self):
        return np.random.randint(0, 5)

    def get_next_bowler(self):
        return np.random.randint(0, 5)

    def get_batting_action(
        self,
        ball,
        total_runs,
        wickets_left,
        score_to_chase,
        feature_batter,
        feature_bowler,
    ):
        batting_action = np.random.randint(0, 6)  # this is the place to code UCTS
        return batting_action

    def get_bowling_action(
        self,
        ball,
        total_runs,
        wickets_left,
        score_to_chase,
        feature_batter,
        feature_bowler,
    ):
        bowling_action = np.random.randint(0, 3)  # this is the place to code UCTS
        return bowling_action

    def explore(self):
        pass


class Australia(Team):
    pass


class India(Team):
    pass

In [ ]:
class Environment:
    def get_outcome(
        self, feature_batter, feature_bowler, batting_action, bowling_action
    ):
        wicket = np.random.randint(0, 1)
        runs = np.random.randint(0, 7)
        return wicket, runs

    get_outcome_random = get_outcome


class Match:
    def __init__(
        self,
        num_balls,
        explore_num_balls,
        action_timeout,
        explore_timeout,
        TeamOne,
        TeamTwo,
    ):
        self.environment = Environment()
        self.num_balls = num_balls
        self.explore_num_balls = explore_num_balls
        self.action_timeout = action_timeout  # time limit for any given act
        self.explore_timeout = explore_timeout  # time limit for exploration phase
        self.team_one_features = np.random.uniform(
            1, 5, size=(5, 4)
        )  # feature[0] : batting average, feature[1]: strike-rate, feature[2]: bowling average, feature[3]: economy
        self.team_two_features = np.random.uniform(1, 5, size=(5, 4))
        self.team_one = TeamOne(
            self.team_one_features, self.team_two_features, self.environment
        )
        self.team_two = TeamTwo(
            self.team_two_features, self.team_one_features, self.environment
        )
        self.current_batters_list = np.array(
            [1, 1, 1, 1, 1]
        )  # a coordinate is set to 0 when that corresponding batter gets out, if the third player is out then you have [1,1,0,1,1]
        self.current_bowlers_list = np.array(
            [2, 2, 2, 2, 2]
        )  # if the 4th bowler bowls the first over, after first over we have [2,2,2,1,2].
        self.num_miss_team_batting = 0
        self.num_miss_team_bowling = 0

    def explore_phase_team(self, team_id):  # timed phase
        if team_id == 1:
            team = self.team_one
        else:
            team = self.team_two
        start_time = time.time()
        team.explore()
        end_time = time.time()
        if end_time - start_time > self.explore_timeout:
            print("Timing Violation During Exploration Phase")

    def explore_phase(self):  # allow each of the teams to explore
        self.explore_phase_team(1)
        self.explore_phase_team(2)

    def get_valid_bowler(self, next_bowler):
        if self.current_bowlers_list[next_bowler] == 0:
            print("Bowler Invalid, Choosing Random Bowler")
            bowlers_with_overs_left = np.where(self.current_bowlers_list > 0)[0]
            next_bowler = np.random.choice(bowlers_with_overs_left)
        return next_bowler

    def get_valid_batter(self, next_batter):
        if self.current_batters_list[next_batter] == 0:
            print("Batter Invalid, Choosing Random Batter")
            batters_not_out = np.where(self.current_batters_list > 0)[0]
            next_batter = np.random.choice(batters_not_out)
        return next_batter

    def next_batter(self, team_batting):
        next_batter = team_batting.get_next_batter()
        next_batter = self.get_valid_batter(next_batter)
        feature_batter = team_batting.self_features[next_batter, 0:2]
        return next_batter, feature_batter

    def next_bowler(self, team_bowling):
        next_bowler = team_bowling.get_next_bowler()
        next_bowler = self.get_valid_bowler(next_bowler)
        feature_bowler = team_bowling.self_features[next_bowler, 2:4]
        return next_bowler, feature_bowler

    def get_team_batting_action(
        self,
        team_batting,
        ball,
        total_runs,
        wickets_left,
        runs_to_chase,
        feature_batter,
        feature_bowler,
    ):
        start_time = time.time()
        batting_action = team_batting.get_batting_action(
            ball,
            total_runs,
            wickets_left,
            runs_to_chase,
            feature_batter,
            feature_bowler,
        )
        end_time = time.time()
        if end_time - start_time > self.action_timeout:
            batting_action = 0  # this is the default option, we have to fix the penalisation strategy
            self.num_miss_team_batting = self.num_miss_team_batting + 1
        return batting_action

    def get_team_bowling_action(
        self,
        team_bowling,
        ball,
        total_runs,
        wickets_left,
        runs_to_chase,
        feature_batter,
        feature_bowler,
    ):
        start_time = time.time()
        bowling_action = team_bowling.get_bowling_action(
            ball,
            total_runs,
            wickets_left,
            runs_to_chase,
            feature_batter,
            feature_bowler,
        )
        end_time = time.time()
        if end_time - start_time > self.action_timeout:
            bowling_action = 0  # this is the default option, we have to fix the penalisation strategy
            self.num_miss_team_bowling = self.num_miss_team_bowling + 1
        return bowling_action

    def innings(self, innigins_id, runs_to_chase):
        total_runs = 0
        wickets_left = 5
        self.current_batters_list = np.array([1, 1, 1, 1, 1])
        self.current_bowlers_list = np.array([2, 2, 2, 2, 2])
        self.num_miss_team_batting = 0
        self.num_miss_team_bowling = 0
        if innigins_id == 1:
            team_batting = self.team_one
            team_bowling = self.team_two
        else:
            team_batting = self.team_two
            team_bowling = self.team_one
        # Initialising the first batter and first bowler
        batter, feature_batter = self.next_batter(team_batting)
        bowler, feature_bowler = self.next_bowler(team_bowling)

        for ball in range(self.num_balls):
            if np.sum(self.current_batters_list) > 0:
                batting_action = self.get_team_batting_action(
                    team_batting,
                    ball,
                    total_runs,
                    wickets_left,
                    runs_to_chase,
                    feature_batter,
                    feature_bowler,
                )
                bowling_action = self.get_team_bowling_action(
                    team_bowling,
                    ball,
                    total_runs,
                    wickets_left,
                    runs_to_chase,
                    feature_batter,
                    feature_bowler,
                )
                wicket, runs = self.environment.get_outcome_random(
                    feature_batter, feature_bowler, batting_action, bowling_action
                )
                total_runs = total_runs + runs
                if wicket > 0:
                    self.current_batters_list[batter] = 0
                    if np.sum(self.current_batters_list) > 0:
                        batter, feature_batter = self.next_batter(team_batting)
                if (ball + 1) % 6 == 0:
                    self.current_bowlers_list[bowler] = (
                        self.current_bowlers_list[bowler] - 1
                    )
                    if np.sum(self.current_bowlers_list) > 0:
                        bowler, feature_bowler = self.next_bowler(team_bowling)

        return (
            total_runs,
            self.current_batters_list,
            self.current_bowlers_list,
            self.num_miss_team_batting,
            self.num_miss_team_bowling,
        )

In [ ]:
import contextlib
from collections import defaultdict
from dataclasses import dataclass, field
from math import sqrt, log
from typing import List, DefaultDict, Callable, Tuple, Dict

import numpy as np

# from cricket_setup import Team

# TODO: Namespace all of this (prevent global variable clashes)


# yuck
tatl_n_actions = 6
tatl_actions = tuple(range(tatl_n_actions))
tatl_actions_set = set(tatl_actions)
tatl_Action = int


# Keys in a python dict need to hashable (and for dataclasses this implies frozen (const))
@dataclass(frozen=True)
class tatl_StateKey:
    balls: int = 0
    target: int = 0


@dataclass
class tatl_State:
    balls: int = 0
    target: int = 0

    def key(self):
        return tatl_StateKey(self.balls, self.target)  # A const version


@dataclass
class tatl_Count:
    n: int = 0  # Total iter
    w: int = 0  # Winning iter / total reward


@dataclass
class tatl_Params:
    count: tatl_Count = field(default_factory=tatl_Count)

    # # Extendable version
    # action_counts: DefaultDict[Action, DefaultDict[StateKey, Count]] = field(
    #     default_factory=lambda: defaultdict(lambda: defaultdict(Count))
    # )

    # Significantly faster version
    action_counts: DefaultDict[tatl_Action, Tuple[tatl_Count, tatl_Count]] = field(
        default_factory=lambda: defaultdict(lambda: (tatl_Count(), tatl_Count()))
    )


# Work with a single rollout
class tatl_Run:
    def __init__(self, policy, get_outcome, go1, go2, go4, target, balls, debug=False):
        self.debug = debug

        self.policy = policy
        self.get_outcome = get_outcome
        self.go1 = go1
        self.go2 = go2
        self.go4 = go4

        self.start_state = tatl_State(balls=balls, target=target)
        self.state = tatl_State(balls=balls, target=target)

        self.wicket = False

        self.state_counts: List[tatl_Count] = []
        self.action_counts: List[tatl_Count] = []

        # Debug:
        self.actions: List[tatl_Action] = []
        # self.targets: List[type(State.target)] = []
        self.targets = []

        if self.debug:
            print(f"Target: {target}\tBalls: {balls}")

    def move(self, move: tatl_Action) -> bool:  # Play a given move
        """
        Takes the given action
        :param move: action to be taken
        :return: Whether further moves can be made
        """
        if (
            self.wicket
            or self.state.target <= 0
            or self.state.target > tatl_n_actions * self.state.balls
        ):
            return False

        # Parent element
        elem: tatl_Params = self.policy.thing[self.state.key()]

        # Apply move
        wicket, runs = self.get_outcome(self.go1, self.go2, move, self.go4)

        if wicket:
            self.wicket = True
            if self.debug:
                print("WICKET!")
            return False

        self.state.target -= runs
        self.state.balls -= 1
        self.actions.append(move)
        self.targets.append(self.state.target)

        # For backpropagation
        state_count = elem.count
        action_count: tatl_Count = elem.action_counts[move][int(runs != 0)]

        self.state_counts.append(state_count)
        self.action_counts.append(action_count)

        if self.state.target <= 0:
            if self.debug:
                print("runs done")
            return False

        if self.state.target > tatl_n_actions * self.state.balls:
            if self.debug:
                print("cant achieve")
            return False

        return True

    def backpropagate(self):
        if self.debug:
            print("Batting action:\t", self.actions)
        if self.debug:
            print("Targets:       \t", self.targets)

        for s_c, a_c in zip(self.state_counts, self.action_counts):
            s_c.n += 1
            a_c.n += 1
        if not self.wicket and self.state.target <= 0:  # If we won
            for s_c, a_c in zip(self.state_counts, self.action_counts):
                s_c.w += 1
                a_c.w += 1


tatl_max_balls_const = 10


# The MCTS tree
class tatl_Policy:
    def __init__(
        self,
        get_outcome: Callable[[tatl_Action], Tuple[bool, int]],
        go1,
        go2,
        go4,
        max_balls: int = tatl_max_balls_const,
    ):
        self.max_balls: int = max_balls

        self.go1 = go1
        self.go2 = go2
        self.go4 = go4
        self.get_outcome = get_outcome

        # The "tree"
        # we arent really using a tree, as we assume that different states can lead to the same next state.
        # this is to reduce the complexity of the 'tree'
        # we have taken a lot of simplifications and approximations when solving the problem
        self.thing: DefaultDict[tatl_StateKey, tatl_Params] = defaultdict(tatl_Params)

    @contextlib.contextmanager
    def do_run(self):
        run = tatl_Run(
            self,
            self.get_outcome,
            self.go1,
            self.go2,
            self.go4,
            target=int(np.random.uniform(tatl_n_actions * self.max_balls)),
            balls=self.max_balls,
        )
        try:
            yield run
        finally:
            run.backpropagate()

    def mcts_choice(self, state: tatl_State) -> tatl_Action:
        elem: tatl_Params = self.thing[state.key()]
        # if elem.count.n != 0:
        #     print(f"%win:\t{elem.count.w * 100 / elem.count.n:.2f}\t{elem.count.n}")

        # diff: set = actions_set - set(elem.action_counts.keys())
        # if len(diff) != 0:
        #     return min(diff)
        for i in tatl_actions:
            if i not in elem.action_counts:
                return i

        mcts_vals: Dict[tatl_Action, float] = {}

        for i, acc in elem.action_counts.items():
            c, c1 = acc

            # # Divide my
            # val = c.n * (c.w / c.n + sqrt(2 * log(elem.count.n) / c.n)) + \
            #       c1.n * (c1.w / c1.n + sqrt(2 * log(elem.count.n) / c1.n))

            val = (
                c.w
                + sqrt(2 * log(elem.count.n) * c.n)
                + c1.w
                + sqrt(2 * log(elem.count.n) * c1.n)
            )
            mcts_vals[i] = val / (c.n + c1.n)

            # # Extendable but computationally expensive (ugh python)
            #     val = sum(map(lambda c: ), acc.values()))
            #     val /= sum(map(lambda c: c.n, acc.values()))
            #
            #     mcts_vals[i] = val

        return max(mcts_vals, key=mcts_vals.get)


class TestTeam(Team):
    # self_features: np.array[5, 4]
    # opponent_features: np.array[5, 4]
    # environment: enviornment.get_outcome(
    #                  feature_batter: np.array[4], feature_bowler: np.array[4],
    #                  batting_action: int, bowling_action: int
    #              ) -> Tuple[int, int]

    def __init__(self, self_features, opponent_features, environment):

        super().__init__(
            self_features, opponent_features, environment
        )  # Call original constructor

        bowling_action = 1

        # saving params (go1, go2, go4) and not a lambda is a 20% speedup
        self.policy = tatl_Policy(
            get_outcome=environment.get_outcome,
            go1=np.mean(self.self_features[:, 0:2], axis=0),
            go2=np.mean(self.opponent_features[:, 2:4], axis=0),
            go4=bowling_action,
        )

    def explore(self):
        # TODO: Implement time constraint
        for i in range(100000):  # ~ 8 seconds
            with self.policy.do_run() as run:
                while run.move(self.policy.mcts_choice(run.state)):
                    pass

    def get_batting_action(
        self,
        ball,
        total_runs,
        wickets_left,
        score_to_chase,
        feature_batter,
        feature_bowler,
    ):
        # TODO: Run more exploration while time is there

        # As the tree only exists to 10 balls, scale it down to that.
        if score_to_chase != float("inf"):
            balls_left = 60 - ball
            target = score_to_chase - total_runs
            if ball + tatl_max_balls_const < 60:
                balls_left = tatl_max_balls_const
                target = target * tatl_max_balls_const / (60 - ball)
                target = int(target)
            # print(balls_left, target, score_to_chase - total_runs, end=' ')
            return self.policy.mcts_choice(tatl_State(balls=balls_left, target=target))
        else:
            magic_constant = 3

            balls_left = 60 - ball
            target = magic_constant * balls_left
            if ball + tatl_max_balls_const < 60:
                balls_left = tatl_max_balls_const
                target = magic_constant * tatl_max_balls_const
                target = int(target)

            # print(balls_left, target, "AA", end=' ')
            return self.policy.mcts_choice(tatl_State(balls=balls_left, target=target))

    def get_next_batter(self):
        # np.max(np.where( self.current_bowlers_list))
        # TODO: Just return batter with max strikerate for now
        # the chance of wicket should ONLY affect batting strategy not batting order
        # Dont have access to ```current_bowlers_list```
        return np.random.randint(0, 5)

In [ ]:
# setting time outs
explore_timeout = 10
action_timeout = 10

explore_num_balls = (
    60  # this number will be much lesser when we actually test the code.
)
num_balls = 60  # fixed
match = Match(
    num_balls, explore_num_balls, action_timeout, explore_timeout, TestTeam, Australia
)
match.explore_phase()
(
    first_innings_score,
    batters_list,
    bowlers_list,
    num_miss_team_batting,
    num_miss_team_bowling,
) = match.innings(1, float("inf"))
(
    second_innings_score,
    batters_list,
    bowlers_list,
    num_miss_team_batting,
    num_miss_team_bowling,
) = match.innings(2, first_innings_score)

Timing Violation During Exploration Phase
Bowler Invalid, Choosing Random Bowler
Bowler Invalid, Choosing Random Bowler
Bowler Invalid, Choosing Random Bowler
Bowler Invalid, Choosing Random Bowler
Bowler Invalid, Choosing Random Bowler
Bowler Invalid, Choosing Random Bowler


In [ ]:
first_innings_score, second_innings_score

(181, 187)